<a href="https://colab.research.google.com/github/glukonatic/III/blob/master/14_Light%2BPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к занятию №14 "Сегментация текста"
## студента Университета искусственного интеллекта
## Наговицына Антона Михайловича
### Поток PRO НС - июль 2020 (основной тариф).

Части Лайт + Про

<img src="https://proza.ru/photos/glukonatic.jpg" alt="Фото Антон Наговицын" width="10%">

#Light

**Light задание. Вариант 1**
 
Сделайте свою собственную архитектуру и запустите, оцените точность на проверочной выборке

Возьмите “лучшую” архитектуру и проведите несколько экспериментов, меняя гиперпараметры выбранной архитектуры.

Напишите ваши выводы


## Загрузка библиотек, объявление функций, подготовка данных

In [1]:
#@title Предустановка недостающей библиотеки

!pip install pymorphy2

     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 8.2MB 12.5MB/s 


In [2]:
#@title Импорт библиотек

import numpy as np
from tensorflow.keras.models import Model, load_model 
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Lambda
from tensorflow.keras import backend as K 
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras import utils 
from google.colab import files 
import matplotlib.pyplot as plt 
import os 
import pandas as pd 
import pymorphy2 
import time

In [3]:
#@title Подключение папки гуглдрайва

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#@title Функции

def readText(fileName):
  f = open(fileName, 'r')
  text = f.read() 
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";"]

  for dS in delSymbols:
    text = text.replace(dS, " ")

  text = re.sub("[.]", " ", text) 
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  text = ' '.join(text.split()) 

  text = text.lower()
  return text


def text2Words(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split(' ')
  words = [morph.parse(word)[0].normal_form for word in words] 
  return words 


def get01XSamples(tok_agreem, tags_index):
  tags01 = [] 
  indexes = [] 
 
  for agreement in tok_agreem: 
    tag_place = [0, 0, 0, 0, 0, 0] 
    for ex in agreement: 
        if ex in tags_index: 
          place = np.argwhere(tags_index==ex) 
          if len(place)!=0: 
            if place[0][0]<6:
              tag_place[place[0][0]] = 1    
            else: 
              tag_place[place[0][0] - 6] = 0  
        else:          
          tags01.append(tag_place.copy())
          indexes.append(ex) 

  return indexes, tags01


def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) 
  words = [reverse_word_map.get(letter) for letter in x] 
  return words 


def getSetFromIndexes(wordIndexes, xLen, step): 
  xBatch = [] 
  wordsLen = len(wordIndexes) 
  index = 0 
  
  while (index + xLen <= wordsLen):
    xBatch.append(wordIndexes[index:index+xLen]) 
    index += step 

  return xBatch 


def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)


def dice_coef_mod(y_true, y_pred, smooth=1):

    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)


def recognizeSet(tagI, pred, tags, length, value):
  total=0

  for j in range(6): 
    correct=0
    for i in range(len(tagI)): 
      for k in range(length): 
        if tagI[i][k][j]==(pred[i][k][j]>value).astype(int):
          correct+=1 
    print("Сеть распознала категорию '{}' на {}%".format(tags[j],round(100*correct/(len(tagI)*length), 2)))
    total += 100 * correct / (len(tagI)*length)
  print("Cредняя точность {}%".format(total/6))


In [5]:
#@title Подготовка данных

directory = '/content/drive/MyDrive/III/14/Договора432/'

agreements = [] 
for filename in os.listdir(directory):
  txt = readText(directory + filename)
  if txt != '':
    agreements.append(readText(directory + filename)) 

words = []
for i in range(len(agreements)): 
  words.append(text2Words(agreements[i])) 
  
wordsToTest = words[-10:] 
words = words[:-10] 

tokenizer = Tokenizer(lower=True, filters = '', char_level=False)
tokenizer.fit_on_texts(words) 

clean_voc = {} 

for item in tokenizer.word_index.items(): 
  clean_voc[item[0]] = item[1] 

tok_agreem = tokenizer.texts_to_sequences(words) 

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] 
closetags = ['</s' + str(i) + '>' for i in range(1, 7)]
tags_index.extend(closetags) 
tags_index = np.array([clean_voc[i] for i in tags_index]) 

xData, yData = get01XSamples(tok_agreem,tags_index) 

decoded_text = reverseIndex(clean_voc, xData)

xLen = 256
step = 30 

xTrain = getSetFromIndexes(decoded_text, xLen, step) 
xTrain = tokenizer.texts_to_sequences(xTrain)
xTrain = np.asarray(xTrain)

yTrain = getSetFromIndexes(yData, xLen, step) 
yTrain = np.array(yTrain)

tok_agreemTest = tokenizer.texts_to_sequences(wordsToTest) 

xDataTest, yDataTest = get01XSamples(tok_agreemTest,tags_index) 

test_decoded_text = reverseIndex(clean_voc, xDataTest) 

xTest = getSetFromIndexes(test_decoded_text, xLen, step) 
xTest = tokenizer.texts_to_sequences(xTest)
xTest = np.asarray(xTest)

yTest = getSetFromIndexes(yDataTest, xLen, step) 
yTest = np.array(yTest)

tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']

## Нейронки

### НС Embedding+Conv1D

In [8]:
#@title Функция создания НС

def create_Conv1d_emb(xLen, embeddingSize): 
  text_input_layer = Input((xLen)) 

  text_layer = Embedding(len(decoded_text), embeddingSize, input_length=xLen)(text_input_layer)
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(16, 3,padding='same', activation='relu')(text_layer) 
  text_layer = Conv1D(yTrain.shape[-1], 3, padding='same',activation='sigmoid')(text_layer)
  model = Model(text_input_layer, text_layer)
  model.compile(optimizer=Adadelta(),
                    loss='categorical_crossentropy',
                    metrics=[dice_coef])
  return model

In [9]:
#@title Создание сети, вывод информации

model_Conv1D = create_Conv1d_emb(xLen, 10)
model_Conv1D.summary() 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 256, 10)           4049990   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 256, 16)           496       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 256, 16)           784       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 256, 6)            294       
Total params: 4,052,348
Trainable params: 4,052,348
Non-trainable params: 0
____________________________________________

In [10]:
#@title Обучение сети

history_Conv1D = model_Conv1D.fit(xTrain, yTrain, epochs=20, batch_size=200, validation_split = 0.2) 

Epoch 1/20
54/54 [==============================] - 1s 13ms/step - loss: 1.2930 - dice_coef: 0.1941 - val_loss: 1.3499 - val_dice_coef: 0.2009
Epoch 2/20
54/54 [==============================] - 0s 8ms/step - loss: 1.2930 - dice_coef: 0.1941 - val_loss: 1.3498 - val_dice_coef: 0.2009
Epoch 3/20
54/54 [==============================] - 0s 8ms/step - loss: 1.2929 - dice_coef: 0.1941 - val_loss: 1.3498 - val_dice_coef: 0.2009
Epoch 4/20
54/54 [==============================] - 0s 8ms/step - loss: 1.2928 - dice_coef: 0.1941 - val_loss: 1.3497 - val_dice_coef: 0.2009
Epoch 5/20
54/54 [==============================] - 0s 8ms/step - loss: 1.2928 - dice_coef: 0.1941 - val_loss: 1.3496 - val_dice_coef: 0.2010
Epoch 6/20
54/54 [==============================] - 0s 8ms/step - loss: 1.2927 - dice_coef: 0.1941 - val_loss: 1.3496 - val_dice_coef: 0.2010
Epoch 7/20
54/54 [==============================] - 0s 8ms/step - loss: 1.2926 - dice_coef: 0.1942 - val_loss: 1.3495 - val_dice_coef: 0.2010
Epoch

In [13]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_Conv1D.predict(xTest), tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 68.71%
Сеть распознала категорию 'S2' на 98.05%
Сеть распознала категорию 'S3' на 87.71%
Сеть распознала категорию 'S4' на 87.39%
Сеть распознала категорию 'S5' на 92.45%
Сеть распознала категорию 'S6' на 99.95%
Cредняя точность 89.04402943586142%


### НС PSPNet+Embedding

In [14]:
#@title Функция создания НС

def create_PSPNet_emb(xLen, embeddingSize, conv_size = 64, num_classes = 6):

    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input((xLen))

    # Block 1
    x = Embedding(len(decoded_text), embeddingSize, input_length=xLen)(img_input)

    x_input = x

    x = Conv1D(conv_size, 3, padding='same')(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(conv_size, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x_mp_2 = MaxPooling1D(2)(x)
    x_mp_4 = MaxPooling1D(4)(x)
    x_mp_8 = MaxPooling1D(8)(x)
    x_mp_16 = MaxPooling1D(16)(x)
    x_mp_32 = MaxPooling1D(32)(x)

    x_mp_2 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_2)
    x_mp_2 = Dropout(0.5)(x_mp_2)

    x_mp_4 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_4)
    x_mp_4 = Dropout(0.5)(x_mp_4)

    x_mp_8 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_8)
    x_mp_8 = Dropout(0.5)(x_mp_8)

    x_mp_16 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_16)
    x_mp_16 = Dropout(0.5)(x_mp_16)

    x_mp_32 = Conv1D(conv_size, 3, padding='same', activation='relu')(x_mp_32)
    x_mp_32 = Dropout(0.5)(x_mp_32) 

    x_mp_2 = Conv1DTranspose(x_mp_2,conv_size, 2)
    x_mp_2 = Activation('relu')(x_mp_2)

    x_mp_4 = Conv1DTranspose(x_mp_4,conv_size, 4, strides=4)
    x_mp_4 = Activation('relu')(x_mp_4)

    x_mp_8 = Conv1DTranspose(x_mp_8,conv_size, 8, strides=8)
    x_mp_8 = Activation('relu')(x_mp_8)

    x_mp_16 = Conv1DTranspose(x_mp_16,conv_size, 16, strides=16)
    x_mp_16 = Activation('relu')(x_mp_16)

    x_mp_32 = Conv1DTranspose(x_mp_32,conv_size, 32, strides=32)
    x_mp_32 = Activation('relu')(x_mp_32)


    fin = concatenate([x_input, x_mp_2, x_mp_4, x_mp_8, x_mp_16, x_mp_32])    
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)
    fin = Conv1D(conv_size, 3, padding='same')(fin)
    fin = BatchNormalization()(fin)
    fin = Activation('relu')(fin)

    fin = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(fin)

    model = Model(img_input, fin)
    model.compile(optimizer=Adadelta(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
    
    return model


In [15]:
#@title Создание сети, вывод информации

model_PSPnet = create_PSPNet_emb(xLen, embeddingSize = 10, conv_size = 256)
model_PSPnet.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 256, 10)      4049990     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 256, 256)     7936        embedding_2[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256)     1024        conv1d_7[0][0]                   
_______________________________________________________________________________________

In [16]:
#@title Обучение сети

history_PSPNet = model_PSPnet.fit(xTrain, yTrain, epochs=60, batch_size=64, validation_split = 0.2)

Epoch 1/60
169/169 [==============================] - 11s 65ms/step - loss: 1.3384 - dice_coef: 0.1973 - val_loss: 1.3401 - val_dice_coef: 0.2007
Epoch 2/60
169/169 [==============================] - 10s 61ms/step - loss: 1.3122 - dice_coef: 0.2027 - val_loss: 1.3459 - val_dice_coef: 0.2003
Epoch 3/60
169/169 [==============================] - 10s 62ms/step - loss: 1.2861 - dice_coef: 0.2088 - val_loss: 1.3254 - val_dice_coef: 0.2074
Epoch 4/60
169/169 [==============================] - 10s 61ms/step - loss: 1.2599 - dice_coef: 0.2152 - val_loss: 1.2994 - val_dice_coef: 0.2158
Epoch 5/60
169/169 [==============================] - 10s 61ms/step - loss: 1.2336 - dice_coef: 0.2221 - val_loss: 1.2797 - val_dice_coef: 0.2217
Epoch 6/60
169/169 [==============================] - 10s 61ms/step - loss: 1.2080 - dice_coef: 0.2292 - val_loss: 1.2592 - val_dice_coef: 0.2273
Epoch 7/60
169/169 [==============================] - 10s 62ms/step - loss: 1.1826 - dice_coef: 0.2364 - val_loss: 1.2392 - 

In [18]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_PSPnet.predict(xTest), tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 68.71%
Сеть распознала категорию 'S2' на 98.05%
Сеть распознала категорию 'S3' на 87.71%
Сеть распознала категорию 'S4' на 87.39%
Сеть распознала категорию 'S5' на 92.45%
Сеть распознала категорию 'S6' на 99.95%
Cредняя точность 89.04402943586142%


### НС UNet+Embedding

In [19]:
#@title Функция создания НС

def create_unet_emb(xLen, embeddingSize = 10, k = 1, num_classes = 6):

    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input(xLen) 

    # Block 1
    x = Embedding(len(decoded_text), embeddingSize, input_length=xLen)(img_input)

    x_input = x

    x = Conv1D(64 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(x, 256, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(x, 128, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(x, 64, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model


In [20]:
#@title Создание сети, вывод информации

model_UNet = create_unet_emb(xLen)
model_UNet.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 256, 10)      4049990     input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 256, 64)      1984        embedding_3[0][0]                
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 256, 64)      256         conv1d_17[0][0]                  
_______________________________________________________________________________________

In [21]:
#@title Обучение НС

history_UNet = model_UNet.fit(xTrain, yTrain, epochs=20, batch_size=64, validation_split = 0.2) 

Epoch 1/20
169/169 [==============================] - 10s 58ms/step - loss: 0.6000 - dice_coef: 0.5048 - val_loss: 7.6931 - val_dice_coef: 0.1683
Epoch 2/20
169/169 [==============================] - 9s 53ms/step - loss: 0.3546 - dice_coef: 0.6305 - val_loss: 7.9154 - val_dice_coef: 0.1084
Epoch 3/20
169/169 [==============================] - 9s 53ms/step - loss: 0.2933 - dice_coef: 0.6711 - val_loss: 5.3996 - val_dice_coef: 0.1671
Epoch 4/20
169/169 [==============================] - 9s 53ms/step - loss: 0.2600 - dice_coef: 0.6986 - val_loss: 2.2070 - val_dice_coef: 0.3701
Epoch 5/20
169/169 [==============================] - 9s 53ms/step - loss: 0.2504 - dice_coef: 0.7095 - val_loss: 1.5232 - val_dice_coef: 0.4812
Epoch 6/20
169/169 [==============================] - 9s 53ms/step - loss: 0.2422 - dice_coef: 0.7223 - val_loss: 1.1040 - val_dice_coef: 0.4939
Epoch 7/20
169/169 [==============================] - 9s 54ms/step - loss: 0.2324 - dice_coef: 0.7328 - val_loss: 1.3773 - val_di

In [22]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_UNet.predict(xTest), tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 69.7%
Сеть распознала категорию 'S2' на 98.06%
Сеть распознала категорию 'S3' на 88.65%
Сеть распознала категорию 'S4' на 87.97%
Сеть распознала категорию 'S5' на 92.24%
Сеть распознала категорию 'S6' на 99.95%
Cредняя точность 89.42898525280897%


### У НС UNet+Embedding меняем гиперпараметры: увеличиваем пространство Embedding-слоя

In [23]:
#@title Создание сети, обучение

model_UNet_100 = create_unet_emb(xLen, embeddingSize=100)
history_UNet_100 = model_UNet_100.fit(xTrain, yTrain, epochs=20, batch_size=64, validation_split = 0.2) 

Epoch 1/20
169/169 [==============================] - 40s 239ms/step - loss: 0.5110 - dice_coef: 0.5489 - val_loss: 3.9077 - val_dice_coef: 0.4112
Epoch 2/20
169/169 [==============================] - 40s 235ms/step - loss: 0.2946 - dice_coef: 0.6682 - val_loss: 2.1819 - val_dice_coef: 0.4626
Epoch 3/20
169/169 [==============================] - 40s 236ms/step - loss: 0.2557 - dice_coef: 0.6993 - val_loss: 1.3079 - val_dice_coef: 0.5177
Epoch 4/20
169/169 [==============================] - 40s 238ms/step - loss: 0.2462 - dice_coef: 0.7081 - val_loss: 1.2727 - val_dice_coef: 0.4996
Epoch 5/20
169/169 [==============================] - 40s 237ms/step - loss: 0.2375 - dice_coef: 0.7196 - val_loss: 2.9225 - val_dice_coef: 0.3379
Epoch 6/20
169/169 [==============================] - 40s 238ms/step - loss: 0.2287 - dice_coef: 0.7321 - val_loss: 1.4482 - val_dice_coef: 0.5259
Epoch 7/20
169/169 [==============================] - 40s 238ms/step - loss: 0.2198 - dice_coef: 0.7407 - val_loss: 2.

In [25]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_UNet_100.predict(xTest), tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 68.86%
Сеть распознала категорию 'S2' на 98.06%
Сеть распознала категорию 'S3' на 87.65%
Сеть распознала категорию 'S4' на 88.01%
Сеть распознала категорию 'S5' на 92.45%
Сеть распознала категорию 'S6' на 99.86%
Cредняя точность 89.14863500702246%


### У НС UNet+Embedding меняем гиперпараметры: увеличиваем пространство Embedding-слоя, увеличиваем число нейронов

In [27]:
#@title Создание сети, обучение

model_UNet_100_2 = create_unet_emb(xLen, embeddingSize=100, k=2)
history_UNet_100_2 = model_UNet_100_2.fit(xTrain, yTrain, epochs=20, batch_size=64, validation_split = 0.2)

Epoch 1/20
169/169 [==============================] - 42s 250ms/step - loss: 0.4766 - dice_coef: 0.5707 - val_loss: 7.7332 - val_dice_coef: 0.1042
Epoch 2/20
169/169 [==============================] - 41s 241ms/step - loss: 0.2809 - dice_coef: 0.6835 - val_loss: 5.5307 - val_dice_coef: 0.1674
Epoch 3/20
169/169 [==============================] - 41s 242ms/step - loss: 0.2508 - dice_coef: 0.7115 - val_loss: 1.8309 - val_dice_coef: 0.4219
Epoch 4/20
169/169 [==============================] - 41s 243ms/step - loss: 0.2372 - dice_coef: 0.7249 - val_loss: 1.3630 - val_dice_coef: 0.5195
Epoch 5/20
169/169 [==============================] - 41s 244ms/step - loss: 0.2252 - dice_coef: 0.7418 - val_loss: 2.0417 - val_dice_coef: 0.5310
Epoch 6/20
169/169 [==============================] - 41s 243ms/step - loss: 0.2192 - dice_coef: 0.7476 - val_loss: 2.8481 - val_dice_coef: 0.3376
Epoch 7/20
169/169 [==============================] - 41s 243ms/step - loss: 0.2354 - dice_coef: 0.7323 - val_loss: 1.

In [29]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_UNet_100_2.predict(xTest), tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 63.26%
Сеть распознала категорию 'S2' на 98.05%
Сеть распознала категорию 'S3' на 89.14%
Сеть распознала категорию 'S4' на 88.0%
Сеть распознала категорию 'S5' на 92.54%
Сеть распознала категорию 'S6' на 99.93%
Cредняя точность 88.48808374297754%


### НС Unet, как в шаге выше, но с модифицированной функцией метрики dice_coef

In [30]:
#@title Функция создания НС

def create_unet_emb_custom(xLen, embeddingSize = 10, k = 1, num_classes = 6):

    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input(xLen) 

    # Block 1
    x = Embedding(len(decoded_text), embeddingSize, input_length=xLen)(img_input)

    x_input = x

    x = Conv1D(64 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(x, 256, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(x, 128, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(x, 64, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef_mod])
  
    return model


In [31]:
#@title Создание сети, обучение

model_UNet_custom = create_unet_emb_custom(xLen, embeddingSize=100, k=2)
history_UNet_custom = model_UNet_custom.fit(xTrain, yTrain, epochs=20, batch_size=64, validation_split = 0.2)

Epoch 1/20
169/169 [==============================] - 42s 248ms/step - loss: 0.4823 - dice_coef_mod: 0.7273 - val_loss: 4.5566 - val_dice_coef_mod: 0.4804
Epoch 2/20
169/169 [==============================] - 41s 244ms/step - loss: 0.2877 - dice_coef_mod: 0.7817 - val_loss: 4.6218 - val_dice_coef_mod: 0.4649
Epoch 3/20
169/169 [==============================] - 41s 244ms/step - loss: 0.2435 - dice_coef_mod: 0.7952 - val_loss: 1.3661 - val_dice_coef_mod: 0.7166
Epoch 4/20
169/169 [==============================] - 41s 243ms/step - loss: 0.2335 - dice_coef_mod: 0.8046 - val_loss: 1.3863 - val_dice_coef_mod: 0.7213
Epoch 5/20
169/169 [==============================] - 41s 243ms/step - loss: 0.2280 - dice_coef_mod: 0.8072 - val_loss: 1.1784 - val_dice_coef_mod: 0.6802
Epoch 6/20
169/169 [==============================] - 41s 244ms/step - loss: 0.2281 - dice_coef_mod: 0.8078 - val_loss: 1.4501 - val_dice_coef_mod: 0.7261
Epoch 7/20
169/169 [==============================] - 41s 244ms/step -

In [32]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_UNet_custom.predict(xTest), tags, xLen, 0.999)


Сеть распознала категорию 'S1' на 70.25%
Сеть распознала категорию 'S2' на 98.03%
Сеть распознала категорию 'S3' на 88.69%
Сеть распознала категорию 'S4' на 88.58%
Сеть распознала категорию 'S5' на 91.65%
Сеть распознала категорию 'S6' на 99.93%
Cредняя точность 89.52206958099252%


## Выводы



1.   При подключении слоя Embedding точность возрасла, хотя и незначительно, по сравнению с моделями нейронок с занятия
2.   Лучше всего себя показала сетка Unet
3.   Так же была написана своя функция рассчета метрики dice_coef, немного отличающейся от заданной на занятии, и это помогло так же немного повысить точность.



# PRO

**Pro задание. Вариант 1**

Используйте другие методы обработки текста (которых не было в занятии, например, embedding слой), поменяйте параметры UNET с занятия и добейтесь с ними точности 85%.


## Загрузка нужных библиотек, объявление функций, подготовка данных

In [36]:
#@title Предустановка недостающей библиотеки

!pip install pymorphy2
!pip install fasttext

In [37]:
#@title Импорт библиотек

import numpy as np
from tensorflow.keras.models import Model, load_model 
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Lambda
from tensorflow.keras import backend as K 
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras import utils 
from google.colab import files 
import fasttext
import matplotlib.pyplot as plt 
import os 
import pandas as pd 
import pymorphy2 
import time

In [38]:
#@title Подключение папки гуглдрайва

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
#@title Функции

def readText(fileName):
  f = open(fileName, 'r')
  text = f.read() 
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";"]

  for dS in delSymbols:
    text = text.replace(dS, " ")

  text = re.sub("[.]", " ", text) 
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  text = ' '.join(text.split()) 

  text = text.lower()
  return text


def text2Words(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split(' ')
  words = [morph.parse(word)[0].normal_form for word in words] 
  return words 


def get01XSamples(tok_agreem, tags_index):
  tags01 = [] 
  indexes = [] 
 
  for agreement in tok_agreem: 
    tag_place = [0, 0, 0, 0, 0, 0] 
    for ex in agreement: 
        if ex in tags_index: 
          place = np.argwhere(tags_index==ex) 
          if len(place)!=0: 
            if place[0][0]<6:
              tag_place[place[0][0]] = 1    
            else: 
              tag_place[place[0][0] - 6] = 0  
        else:          
          tags01.append(tag_place.copy())
          indexes.append(ex) 

  return indexes, tags01


def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) 
  words = [reverse_word_map.get(letter) for letter in x] 
  return words 


def getSetFromIndexes(wordIndexes, xLen, step): 
  xBatch = [] 
  wordsLen = len(wordIndexes) 
  index = 0 
  
  while (index + xLen <= wordsLen):
    xBatch.append(wordIndexes[index:index+xLen]) 
    index += step 

  return xBatch 


def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)


def dice_coef_mod(y_true, y_pred, smooth=1):

    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)


def recognizeSet(tagI, pred, tags, length, value):
  total=0

  for j in range(6): 
    correct=0
    for i in range(len(tagI)): 
      for k in range(length): 
        if tagI[i][k][j]==(pred[i][k][j]>value).astype(int):
          correct+=1 
    print("Сеть распознала категорию '{}' на {}%".format(tags[j],round(100*correct/(len(tagI)*length), 2)))
    total += 100 * correct / (len(tagI)*length)
  print("Cредняя точность {}%".format(total/6))


In [40]:
#@title Подготовка данных

directory = '/content/drive/MyDrive/III/14/Договора432/'

agreements = [] 
for filename in os.listdir(directory):
  txt = readText(directory + filename)
  if txt != '':
    agreements.append(readText(directory + filename)) 

words = []
for i in range(len(agreements)): 
  words.append(text2Words(agreements[i])) 
  
wordsToTest = words[-10:] 
words = words[:-10] 

tokenizer = Tokenizer(lower=True, filters = '', char_level=False)
tokenizer.fit_on_texts(words) 

clean_voc = {} 

for item in tokenizer.word_index.items(): 
  clean_voc[item[0]] = item[1] 

tok_agreem = tokenizer.texts_to_sequences(words) 

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] 
closetags = ['</s' + str(i) + '>' for i in range(1, 7)]
tags_index.extend(closetags) 
tags_index = np.array([clean_voc[i] for i in tags_index]) 

xData, yData = get01XSamples(tok_agreem,tags_index) 

decoded_text = reverseIndex(clean_voc, xData)

xLen = 256
step = 30 

xTrain = getSetFromIndexes(decoded_text, xLen, step) 
xTrain = tokenizer.texts_to_sequences(xTrain)
xTrain = np.asarray(xTrain)

yTrain = getSetFromIndexes(yData, xLen, step) 
yTrain = np.array(yTrain)

tok_agreemTest = tokenizer.texts_to_sequences(wordsToTest) 

xDataTest, yDataTest = get01XSamples(tok_agreemTest,tags_index) 

test_decoded_text = reverseIndex(clean_voc, xDataTest) 

xTest = getSetFromIndexes(test_decoded_text, xLen, step) 
xTest = tokenizer.texts_to_sequences(xTest)
xTest = np.asarray(xTest)

yTest = getSetFromIndexes(yDataTest, xLen, step) 
yTest = np.array(yTest)

tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [85]:
#@title Дополнительная подготовка для работы с матрицей весов fasttext
#@markdown https://fasttext.cc/

f = open('text.txt', 'w')

for text in agreements:
  f.write(text)

f.close()

embeddings_index = fasttext.train_unsupervised('text.txt', model='cbow')

embedding_matrix = np.zeros((len(decoded_text), 100))

for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index[word]
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

## Нейронки

### НС Embedding(fasttext)+Conv1D

In [106]:
#@title Функция создания НС

def create_Conv1d_emb_fasttext(xLen, embeddingSize): 
  text_input_layer = Input((xLen)) 

  text_layer = Embedding(len(decoded_text), embeddingSize, weights=[embedding_matrix], input_length=xLen, trainable=False)(text_input_layer)

  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(16, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(16, 3,padding='same', activation='relu')(text_layer) 
  text_layer = Conv1D(yTrain.shape[-1], 3, padding='same',activation='sigmoid')(text_layer)
  model = Model(text_input_layer, text_layer)
  model.compile(optimizer=Adadelta(),
                    loss='categorical_crossentropy',
                    metrics=[dice_coef])
  return model

In [108]:
#@title Создание сети, вывод информации

model_Conv1D_fasttext = create_Conv1d_emb_fasttext(xLen, 100)
model_Conv1D_fasttext.summary() 

Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 256)]             0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 256, 100)          40499900  
_________________________________________________________________
conv1d_102 (Conv1D)          (None, 256, 16)           4816      
_________________________________________________________________
conv1d_103 (Conv1D)          (None, 256, 16)           784       
_________________________________________________________________
conv1d_104 (Conv1D)          (None, 256, 16)           784       
_________________________________________________________________
conv1d_105 (Conv1D)          (None, 256, 6)            294       
Total params: 40,506,578
Trainable params: 6,678
Non-trainable params: 40,499,900
_____________________________________

In [109]:
#@title Обучение сети

history_Conv1D_fasttext = model_Conv1D_fasttext.fit(xTrain, yTrain, epochs=20, batch_size=200, validation_split = 0.2) 

Epoch 1/20
54/54 [==============================] - 1s 12ms/step - loss: 1.2430 - dice_coef: 0.2089 - val_loss: 1.3090 - val_dice_coef: 0.2123
Epoch 2/20
54/54 [==============================] - 0s 9ms/step - loss: 1.2420 - dice_coef: 0.2091 - val_loss: 1.3081 - val_dice_coef: 0.2126
Epoch 3/20
54/54 [==============================] - 0s 9ms/step - loss: 1.2409 - dice_coef: 0.2095 - val_loss: 1.3072 - val_dice_coef: 0.2128
Epoch 4/20
54/54 [==============================] - 0s 9ms/step - loss: 1.2397 - dice_coef: 0.2098 - val_loss: 1.3062 - val_dice_coef: 0.2131
Epoch 5/20
54/54 [==============================] - 0s 9ms/step - loss: 1.2384 - dice_coef: 0.2102 - val_loss: 1.3051 - val_dice_coef: 0.2134
Epoch 6/20
54/54 [==============================] - 0s 9ms/step - loss: 1.2371 - dice_coef: 0.2105 - val_loss: 1.3040 - val_dice_coef: 0.2137
Epoch 7/20
54/54 [==============================] - 0s 9ms/step - loss: 1.2357 - dice_coef: 0.2110 - val_loss: 1.3028 - val_dice_coef: 0.2140
Epoch

In [110]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_Conv1D_fasttext.predict(xTest), tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 68.71%
Сеть распознала категорию 'S2' на 98.05%
Сеть распознала категорию 'S3' на 87.71%
Сеть распознала категорию 'S4' на 87.39%
Сеть распознала категорию 'S5' на 92.45%
Сеть распознала категорию 'S6' на 99.95%
Cредняя точность 89.04402943586142%


### НС Embedding(fasttext)+UNet+модифицированная функция метрики dice_coef

In [111]:
#@title Функция создания НС

def create_unet_fasttext(xLen, embeddingSize = 10, k = 1, num_classes = 6):

    def Conv1DTranspose(input_tensor, filters, kernel_size=1, strides=2, padding='same'):
        """
        input_tensor: входной тензор (batch_size, time_steps, dims)
        filters: int, output dimension, выходной тензор будет иметь размер (batch_size, time_steps, filters)
        kernel_size: размер ядра свертки 
        strides: int, шаг ядра
        padding: 'same' | действительный
        """
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x
        
    img_input = Input(xLen) 

    # Block 1
    x = Embedding(len(decoded_text), embeddingSize, weights=[embedding_matrix], input_length=xLen, trainable=False)(img_input)

    x_input = x

    x = Conv1D(64 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(x, 256, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(x, 128, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(x, 64, kernel_size=2, strides=2, padding='same')
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef_mod])
  
    return model


In [113]:
#@title Создание сети, обучение

model_UNet_fasttext = create_unet_fasttext(xLen, embeddingSize=100, k=2)
history_UNet_fasttext = model_UNet_fasttext.fit(xTrain, yTrain, epochs=20, batch_size=64, validation_split = 0.2)

Epoch 1/20
169/169 [==============================] - 14s 83ms/step - loss: 0.6412 - dice_coef_mod: 0.6855 - val_loss: 2.4978 - val_dice_coef_mod: 0.5624
Epoch 2/20
169/169 [==============================] - 14s 80ms/step - loss: 0.4339 - dice_coef_mod: 0.7441 - val_loss: 1.6146 - val_dice_coef_mod: 0.6822
Epoch 3/20
169/169 [==============================] - 14s 80ms/step - loss: 0.3229 - dice_coef_mod: 0.7700 - val_loss: 1.5521 - val_dice_coef_mod: 0.6941
Epoch 4/20
169/169 [==============================] - 14s 80ms/step - loss: 0.2746 - dice_coef_mod: 0.7845 - val_loss: 1.4757 - val_dice_coef_mod: 0.7114
Epoch 5/20
169/169 [==============================] - 14s 80ms/step - loss: 0.2488 - dice_coef_mod: 0.7940 - val_loss: 1.2442 - val_dice_coef_mod: 0.6985
Epoch 6/20
169/169 [==============================] - 14s 80ms/step - loss: 0.2431 - dice_coef_mod: 0.7993 - val_loss: 1.2059 - val_dice_coef_mod: 0.7110
Epoch 7/20
169/169 [==============================] - 14s 80ms/step - loss: 

In [114]:
#@title Результаты обучения: Качество модели

recognizeSet(yTest, model_UNet_fasttext.predict(xTest), tags, xLen, 0.999)


Сеть распознала категорию 'S1' на 74.64%
Сеть распознала категорию 'S2' на 98.05%
Сеть распознала категорию 'S3' на 88.41%
Сеть распознала категорию 'S4' на 87.61%
Сеть распознала категорию 'S5' на 92.45%
Сеть распознала категорию 'S6' на 99.95%
Cредняя точность 90.18591262874531%


# Конец